# AlphaGenome Quick Start

This notebook mirrors the quick start pipeline and reads the API key from the local `.env` file so nothing sensitive is hard-coded.

In [ ]:
import os

from dotenv import load_dotenv
from IPython.display import clear_output

from alphagenome import colab_utils
from alphagenome.data import gene_annotation
from alphagenome.data import genome
from alphagenome.data import transcript as transcript_utils
from alphagenome.interpretation import ism
from alphagenome.models import dna_client
from alphagenome.models import variant_scorers
from alphagenome.visualization import plot_components
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
load_dotenv()
clear_output()

api_key = os.getenv('ALPHA_GENOME_API_KEY') or os.getenv('ALPHA_GENOME_API_KEY')
if not api_key:
    raise RuntimeError('Missing ALPHA_GENOME_KEY in environment. Check the .env file.')

dna_model = dna_client.create(api_key)
dna_client.OutputType

## Predict outputs for a DNA sequence

AlphaGenome makes predictions from DNA sequences. This example pads a short sequence to 1 Mbp and requests DNase tracks for lung tissue.

In [ ]:
output = dna_model.predict_sequence(
    sequence='GATTACA'.center(dna_client.SEQUENCE_LENGTH_1MB, 'N'),
    requested_outputs=[dna_client.OutputType.DNASE],
    ontology_terms=['UBERON:0002048'],
)

dnase = output.dnase
dnase.values.shape

In [ ]:
dnase.values

In [ ]:
dnase.metadata

Requesting multiple assays or tissues at once returns additional tracks and metadata.

In [ ]:
output_multi = dna_model.predict_sequence(
    sequence='GATTACA'.center(dna_client.SEQUENCE_LENGTH_1MB, 'N'),
    requested_outputs=[
        dna_client.OutputType.CAGE,
        dna_client.OutputType.DNASE,
    ],
    ontology_terms=[
        'UBERON:0002048',
        'UBERON:0000955',
    ],
)

print(f'DNASE predictions shape: {output_multi.dnase.values.shape}')
print(f'CAGE predictions shape: {output_multi.cage.values.shape}')
output_multi.cage.metadata

## Predict outputs for a genome interval

Load annotations, extract a genomic interval around *CYP2B6*, resize it to a supported length, and request RNA-seq predictions for liver tissue.

In [ ]:
gtf = pd.read_feather(
    'https://storage.googleapis.com/alphagenome/reference/gencode/hg38/gencode.v46.annotation.gtf.gz.feather'
)
gtf_transcripts = gene_annotation.filter_protein_coding(gtf)
gtf_transcripts = gene_annotation.filter_to_longest_transcript(gtf_transcripts)
transcript_extractor = transcript_utils.TranscriptExtractor(gtf_transcripts)

In [ ]:
interval = gene_annotation.get_gene_interval(gtf, gene_symbol='CYP2B6')
interval = interval.resize(dna_client.SEQUENCE_LENGTH_1MB)
interval

In [ ]:
output_interval = dna_model.predict_interval(
    interval=interval,
    requested_outputs=[dna_client.OutputType.RNA_SEQ],
    ontology_terms=['UBERON:0001114'],
)

output_interval.rna_seq.values.shape

In [ ]:
longest_transcripts = transcript_extractor.extract(interval)
print(f'Extracted {len(longest_transcripts)} transcripts in this interval.')
plot_components.plot(
    components=[
        plot_components.TranscriptAnnotation(longest_transcripts),
        plot_components.Tracks(output_interval.rna_seq),
    ],
    interval=output_interval.rna_seq.interval,
)
plt.show()

In [ ]:
plot_components.plot(
    components=[
        plot_components.TranscriptAnnotation(longest_transcripts, fig_height=0.1),
        plot_components.Tracks(output_interval.rna_seq),
    ],
    interval=output_interval.rna_seq.interval.resize(2**15),
)
plt.show()

## Predict variant effects

Score the effect of a single variant on RNA-seq predictions for colon tissue and visualize the result.

In [ ]:
variant = genome.Variant(
    chromosome='chr22',
    position=36201698,
    reference_bases='A',
    alternate_bases='C',
)
variant_interval = variant.reference_interval.resize(dna_client.SEQUENCE_LENGTH_1MB)
variant_interval

In [ ]:
variant_output = dna_model.predict_variant(
    interval=variant_interval,
    variant=variant,
    requested_outputs=[dna_client.OutputType.RNA_SEQ],
    ontology_terms=['UBERON:0001157'],
)

longest_transcripts_variant = transcript_extractor.extract(variant_interval)
plot_components.plot(
    [
        plot_components.TranscriptAnnotation(longest_transcripts_variant),
        plot_components.OverlaidTracks(
            tdata={
                'REF': variant_output.reference.rna_seq,
                'ALT': variant_output.alternate.rna_seq,
            },
            colors={'REF': 'dimgrey', 'ALT': 'red'},
        ),
    ],
    interval=variant_output.reference.rna_seq.interval.resize(2**15),
    annotations=[plot_components.VariantAnnotation([variant], alpha=0.8)],
)
plt.show()

## Score the effect of a genetic variant

Use a recommended variant scorer to aggregate REF/ALT predictions into gene-level scores.

In [ ]:
variant_scorer = variant_scorers.RECOMMENDED_VARIANT_SCORERS['RNA_SEQ']
variant_scores = dna_model.score_variant(
    interval=variant_interval,
    variant=variant,
    variant_scorers=[variant_scorer],
)
len(variant_scores)

In [ ]:
variant_scores = variant_scores[0]
print(variant_scores)
print(variant_scores.X.shape)
variant_scores.obs.head()

In [ ]:
variant_scorers.tidy_scores([variant_scores], match_gene_strand=True)

## In silico mutagenesis (ISM)

Score all single-base substitutions within a 256 bp ISM window using a DNase aggregation scorer and visualize the results.

In [ ]:
sequence_interval = genome.Interval('chr20', 3_753_000, 3_753_400)
sequence_interval = sequence_interval.resize(dna_client.SEQUENCE_LENGTH_2KB)
ism_interval = sequence_interval.resize(256)
sequence_interval, ism_interval

In [ ]:
dnase_variant_scorer = variant_scorers.CenterMaskScorer(
    requested_output=dna_client.OutputType.DNASE,
    width=501,
    aggregation_type=variant_scorers.AggregationType.DIFF_MEAN,
)
dnase_variant_scorer

In [ ]:
variant_scores_ism = dna_model.score_ism_variants(
    interval=sequence_interval,
    ism_interval=ism_interval,
    variant_scorers=[dnase_variant_scorer],
)
len(variant_scores_ism)

In [ ]:
variant_scores_ism[0][0].X.shape

In [ ]:
def extract_k562(adata):
    values = adata.X[:, adata.var['ontology_curie'] == 'EFO:0002067']
    assert values.size == 1
    return values.flatten()[0]

ism_result = ism.ism_matrix(
    [extract_k562(x[0]) for x in variant_scores_ism],
    variants=[v[0].uns['variant'] for v in variant_scores_ism],
)
ism_result.shape

In [ ]:
plot_components.plot(
    [
        plot_components.SeqLogo(
            scores=ism_result,
            scores_interval=ism_interval,
            ylabel='ISM K562 DNase',
        )
    ],
    interval=ism_interval,
    fig_width=35,
)
plt.show()

That's the complete quick start workflow. Adjust ontology terms, organisms, and scorers as needed for your analyses.